# Implementación Naive Bayes y Logistic Regression

## Configuraciones iniciales

In [1]:
# Imports básicos
from pathlib import Path
from typing import Tuple, Dict, Any
import numpy as np
import pandas as pd

# Sklearn imports
from sklearn.datasets import load_files
from sklearn.model_selection import train_test_split, cross_validate, StratifiedKFold, GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import (classification_report, precision_recall_fscore_support, 
                            confusion_matrix, accuracy_score)

print("Todos los imports cargados correctamente")

Todos los imports cargados correctamente


Para el split de los datos se definieron los porcentajes de acorde a lo establecido por el enunciado.

In [3]:
DATA_DIR = Path(r"Datasets/20news-18828/20news-18828/")  # AJUSTAR SEGÚN TU SISTEMA
RANDOM_STATE = 42
TEST_SIZE = 0.30           # 30% para test
VAL_RATIO_WITHIN_TRAINVAL = 1.0 / 7.0  # 10% absoluto para validación (de 70% restante)
USE_ENGLISH_STOPWORDS = True

print("Configuración establecida")
print(f"   - Test size: {TEST_SIZE*100}%")
print(f"   - Validation size: {VAL_RATIO_WITHIN_TRAINVAL * (1-TEST_SIZE) * 100:.1f}%")
print(f"   - Training size: {(1-TEST_SIZE) * (1-VAL_RATIO_WITHIN_TRAINVAL) * 100:.1f}%")

Configuración establecida
   - Test size: 30.0%
   - Validation size: 10.0%
   - Training size: 60.0%


## Carga de datos y train_val_test script

### 20News

Se cargan los datos utilizando el encoding latin-1 para no tener problemas de codificación para algunos de los caracteres presentes en el dataset.

In [4]:
def cargar_dataset(dataroot: Path):
    """
    Carga el dataset 20newsgroups desde archivos organizados en subcarpetas.
    """
    dataset = load_files(
        container_path=str(dataroot),
        encoding="latin-1",
        decode_error="ignore",
        shuffle=True,
        random_state=RANDOM_STATE
    )
    return dataset

Se realiza la partición de los datasets.

In [5]:
def partir_train_val_test(X, y, test_size: float, val_ratio_within_trainval: float, random_state: int):
    """
    Crea partición 60/10/30 estratificada.
    """
    # Primero: (train+val)=70% y test=30%
    X_trainval, X_test, y_trainval, y_test = train_test_split(
        X, y, test_size=test_size, random_state=random_state, stratify=y
    )
    
    # Luego: dentro del 70%, separa val=10% absoluto y train=60% absoluto
    X_train, X_val, y_train, y_val = train_test_split(
        X_trainval, y_trainval,
        test_size=val_ratio_within_trainval,
        random_state=random_state,
        stratify=y_trainval
    )
    
    return X_train, y_train, X_val, y_val, X_test, y_test

Se construyó una función que permitiera construir un vectorizador que permitiera devolver el vectorizador a utilizar basado en un parámetro.

In [6]:
def construir_vectorizador(kind: str, use_english_stopwords: bool, optimized=True):
    """
    Construye vectorizador con parámetros optimizados o básicos.
    """
    stop_words = "english" if use_english_stopwords else None
    
    if optimized:
        # Parámetros menos restrictivos para mejor rendimiento
        min_df = 2
        max_df = 0.95
        max_features = 50000
    else:
        # Parámetros más restrictivos para velocidad
        min_df = 5
        max_df = 0.90
        max_features = 20000
    
    if kind == "tf":
        return CountVectorizer(
            stop_words=stop_words,
            min_df=min_df,
            max_df=max_df,
            max_features=max_features
        )
    elif kind == "tfidf":
        return TfidfVectorizer(
            stop_words=stop_words,
            min_df=min_df,
            max_df=max_df,
            max_features=max_features
        )
    else:
        raise ValueError("kind debe ser 'tf' o 'tfidf'")

Para la evaluación se realizó una función que realiza las siguientes acciones:

1. Entrena utilizando únicamente el dataset de train y se evalúa con el conjunto de validación.
2. Luego de haber calculado estas métricas se reentrena el modelo pero esta vez utilizando ambos conjuntos de datos (entrenamiento y validación).
3. Este nuevo modelo se evalúa contra el dataset de test y se vuelve a calcular métricas.

In [ ]:
def evaluar_modelo_simple(pipeline, X_train, y_train, X_val, y_val, X_test, y_test, 
                         nombre_modelo, target_names, mostrar_detalles=True):
    """
    Entrena y evalúa un modelo en validación y test.
    """
    # Entrenar solo en train
    pipeline.fit(X_train, y_train)
    
    # Evaluar en validación
    y_pred_val = pipeline.predict(X_val)
    acc_val = accuracy_score(y_val, y_pred_val)
    p_mac_val, r_mac_val, f1_mac_val, _ = precision_recall_fscore_support(y_val, y_pred_val, average="macro")
    p_mic_val, r_mic_val, f1_mic_val, _ = precision_recall_fscore_support(y_val, y_pred_val, average="micro")
    
    # Re-entrenar con train+val para test
    X_train_full = np.concatenate([X_train, X_val])
    y_train_full = np.concatenate([y_train, y_val])
    pipeline.fit(X_train_full, y_train_full)
    
    # Evaluar en test
    y_pred_test = pipeline.predict(X_test)
    acc_test = accuracy_score(y_test, y_pred_test)
    p_mac_test, r_mac_test, f1_mac_test, _ = precision_recall_fscore_support(y_test, y_pred_test, average="macro")
    p_mic_test, r_mic_test, f1_mic_test, _ = precision_recall_fscore_support(y_test, y_pred_test, average="micro")
    
    if mostrar_detalles:
        print(f"\n{'='*80}")
        print(f"MODELO: {nombre_modelo}")
        print(f"{'='*80}")
        
        print("\n VALIDACIÓN:")
        print(f"   Accuracy: {acc_val:.4f}")
        print(f"   Macro  -> P: {p_mac_val:.4f}  R: {r_mac_val:.4f}  F1: {f1_mac_val:.4f}")
        print(f"   Micro  -> P: {p_mic_val:.4f}  R: {r_mic_val:.4f}  F1: {f1_mic_val:.4f}")
        
        print("\n TEST:")
        print(f"   Accuracy: {acc_test:.4f}")
        print(f"   Macro  -> P: {p_mac_test:.4f}  R: {r_mac_test:.4f}  F1: {f1_mac_test:.4f}")
        print(f"   Micro  -> P: {p_mic_test:.4f}  R: {r_mic_test:.4f}  F1: {f1_mic_test:.4f}")
        
        print("\n Classification Report (Test):")
        print(classification_report(y_test, y_pred_test, target_names=target_names, digits=3))
    
    return {
        'val_accuracy': acc_val, 'val_f1_macro': f1_mac_val, 'val_f1_micro': f1_mic_val,
        'test_accuracy': acc_test, 'test_f1_macro': f1_mac_test, 'test_f1_micro': f1_mic_test
    }


## Ejecución de la carga, ejecución y evaluación.

In [8]:
print(" Cargando dataset 20newsgroups...")
dataset = cargar_dataset(DATA_DIR)

X = dataset.data
y = dataset.target
target_names = dataset.target_names

print(f" Dataset cargado:")
print(f"   - Total documentos: {len(X):,}")
print(f"   - Número de clases: {len(target_names)}")
print(f"   - Primeras 5 clases: {target_names[:5]}")

 Cargando dataset 20newsgroups...
 Dataset cargado:
   - Total documentos: 18,828
   - Número de clases: 20
   - Primeras 5 clases: ['alt.atheism', 'comp.graphics', 'comp.os.ms-windows.misc', 'comp.sys.ibm.pc.hardware', 'comp.sys.mac.hardware']


In [11]:
print("\n Creando particiones de datos...")
X_train, y_train, X_val, y_val, X_test, y_test = partir_train_val_test(
    X, y, TEST_SIZE, VAL_RATIO_WITHIN_TRAINVAL, RANDOM_STATE
)

print(f" Particiones creadas:")
print(f"   - Train: {len(X_train):,} documentos ({len(X_train)/len(X)*100:.1f}%)")
print(f"   - Val:   {len(X_val):,} documentos ({len(X_val)/len(X)*100:.1f}%)")
print(f"   - Test:  {len(X_test):,} documentos ({len(X_test)/len(X)*100:.1f}%)")


 Creando particiones de datos...
 Particiones creadas:
   - Train: 11,296 documentos (60.0%)
   - Val:   1,883 documentos (10.0%)
   - Test:  5,649 documentos (30.0%)


### Construcción de los pipelines de entrenamiento

Una vez se han definido todas lasfunciones requeridas para este entrenamiento el siguiente paso es definir los pipelines para realizar los entrenamientos para todas las combinaciones necesarias entre los modelos de clasificación y el vectorizador a utilizar.

In [12]:
print(f"\n{'='*80}")
print("I. COMPARACIÓN DE CLASIFICADORES NB Y LR")
print(f"{'='*80}")

print("\n Entrenando modelos con representaciones TF y TF-IDF...")

# Crear pipelines
pipelines = {
    "NB + TF": Pipeline([
        ("vec", construir_vectorizador("tf", USE_ENGLISH_STOPWORDS, optimized=True)),
        ("clf", MultinomialNB())
    ]),
    "NB + TF-IDF": Pipeline([
        ("vec", construir_vectorizador("tfidf", USE_ENGLISH_STOPWORDS, optimized=True)),
        ("clf", MultinomialNB())
    ]),
    "LR + TF": Pipeline([
        ("vec", construir_vectorizador("tf", USE_ENGLISH_STOPWORDS, optimized=True)),
        ("clf", LogisticRegression(solver="liblinear", multi_class="ovr", random_state=RANDOM_STATE, max_iter=2000))
    ]),
    "LR + TF-IDF": Pipeline([
        ("vec", construir_vectorizador("tfidf", USE_ENGLISH_STOPWORDS, optimized=True)),
        ("clf", LogisticRegression(solver="liblinear", multi_class="ovr", random_state=RANDOM_STATE, max_iter=2000))
    ])
}

# Evaluar cada pipeline
resultados_parte1 = {}
for nombre, pipeline in pipelines.items():
    print(f"\n  Entrenando {nombre}...")
    resultado = evaluar_modelo_simple(
        pipeline, X_train, y_train, X_val, y_val, X_test, y_test,
        nombre, target_names, mostrar_detalles=True
    )
    resultados_parte1[nombre] = resultado


I. COMPARACIÓN DE CLASIFICADORES NB Y LR

 Entrenando modelos con representaciones TF y TF-IDF...

  Entrenando NB + TF...

MODELO: NB + TF

 VALIDACIÓN:
   Accuracy: 0.8640
   Macro  -> P: 0.8813  R: 0.8617  F1: 0.8485
   Micro  -> P: 0.8640  R: 0.8640  F1: 0.8640

 TEST:
   Accuracy: 0.8727
   Macro  -> P: 0.8833  R: 0.8694  F1: 0.8610
   Micro  -> P: 0.8727  R: 0.8727  F1: 0.8727

 Classification Report (Test):
                          precision    recall  f1-score   support

             alt.atheism      0.875     0.938     0.905       240
           comp.graphics      0.700     0.856     0.770       292
 comp.os.ms-windows.misc      0.902     0.186     0.308       296
comp.sys.ibm.pc.hardware      0.638     0.861     0.733       295
   comp.sys.mac.hardware      0.831     0.906     0.867       288
          comp.windows.x      0.743     0.915     0.820       294
            misc.forsale      0.881     0.812     0.845       292
               rec.autos      0.922     0.916     0.

c:\ProgramData\anaconda3\envs\ir-gensim\lib\site-packages\sklearn\linear_model\_logistic.py:1281: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.8. Use OneVsRestClassifier(LogisticRegression(..)) instead. Leave it to its default value to avoid this warning.
  warnings.warn(
c:\ProgramData\anaconda3\envs\ir-gensim\lib\site-packages\sklearn\linear_model\_logistic.py:1296: FutureWarning: Using the 'liblinear' solver for multiclass classification is deprecated. An error will be raised in 1.8. Either use another solver which supports the multinomial loss or wrap the estimator in a OneVsRestClassifier to keep applying a one-versus-rest scheme.
  warnings.warn(
c:\ProgramData\anaconda3\envs\ir-gensim\lib\site-packages\sklearn\linear_model\_logistic.py:1281: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.8. Use OneVsRestClassifier(LogisticRegression(..)) instead. Leave it to its default value to avoid this warning.
  war


MODELO: LR + TF

 VALIDACIÓN:
   Accuracy: 0.9028
   Macro  -> P: 0.9055  R: 0.9015  F1: 0.9028
   Micro  -> P: 0.9028  R: 0.9028  F1: 0.9028

 TEST:
   Accuracy: 0.8986
   Macro  -> P: 0.8990  R: 0.8960  F1: 0.8971
   Micro  -> P: 0.8986  R: 0.8986  F1: 0.8986

 Classification Report (Test):
                          precision    recall  f1-score   support

             alt.atheism      0.903     0.896     0.900       240
           comp.graphics      0.818     0.849     0.834       292
 comp.os.ms-windows.misc      0.838     0.841     0.840       296
comp.sys.ibm.pc.hardware      0.787     0.776     0.782       295
   comp.sys.mac.hardware      0.846     0.896     0.870       288
          comp.windows.x      0.866     0.857     0.862       294
            misc.forsale      0.821     0.880     0.850       292
               rec.autos      0.913     0.919     0.916       297
         rec.motorcycles      0.950     0.956     0.953       298
      rec.sport.baseball      0.950     0.95

c:\ProgramData\anaconda3\envs\ir-gensim\lib\site-packages\sklearn\linear_model\_logistic.py:1281: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.8. Use OneVsRestClassifier(LogisticRegression(..)) instead. Leave it to its default value to avoid this warning.
  warnings.warn(
c:\ProgramData\anaconda3\envs\ir-gensim\lib\site-packages\sklearn\linear_model\_logistic.py:1296: FutureWarning: Using the 'liblinear' solver for multiclass classification is deprecated. An error will be raised in 1.8. Either use another solver which supports the multinomial loss or wrap the estimator in a OneVsRestClassifier to keep applying a one-versus-rest scheme.
  warnings.warn(
c:\ProgramData\anaconda3\envs\ir-gensim\lib\site-packages\sklearn\linear_model\_logistic.py:1281: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.8. Use OneVsRestClassifier(LogisticRegression(..)) instead. Leave it to its default value to avoid this warning.
  war


MODELO: LR + TF-IDF

 VALIDACIÓN:
   Accuracy: 0.8996
   Macro  -> P: 0.9006  R: 0.8944  F1: 0.8958
   Micro  -> P: 0.8996  R: 0.8996  F1: 0.8996

 TEST:
   Accuracy: 0.8940
   Macro  -> P: 0.8963  R: 0.8878  F1: 0.8897
   Micro  -> P: 0.8940  R: 0.8940  F1: 0.8940

 Classification Report (Test):
                          precision    recall  f1-score   support

             alt.atheism      0.887     0.879     0.883       240
           comp.graphics      0.790     0.839     0.814       292
 comp.os.ms-windows.misc      0.821     0.865     0.842       296
comp.sys.ibm.pc.hardware      0.799     0.769     0.784       295
   comp.sys.mac.hardware      0.896     0.899     0.898       288
          comp.windows.x      0.891     0.888     0.889       294
            misc.forsale      0.823     0.873     0.847       292
               rec.autos      0.895     0.919     0.907       297
         rec.motorcycles      0.962     0.940     0.951       298
      rec.sport.baseball      0.960     

In [13]:
print(f"\n{'='*80}")
print(" RESUMEN PARTE I - COMPARACIÓN INICIAL")
print(f"{'='*80}")

df_parte1 = pd.DataFrame(resultados_parte1).T
print("\n Métricas en TEST:")
print(df_parte1[['test_accuracy', 'test_f1_macro', 'test_f1_micro']].round(4))


 RESUMEN PARTE I - COMPARACIÓN INICIAL

 Métricas en TEST:
             test_accuracy  test_f1_macro  test_f1_micro
NB + TF             0.8727         0.8610         0.8727
NB + TF-IDF         0.8911         0.8801         0.8911
LR + TF             0.8986         0.8971         0.8986
LR + TF-IDF         0.8940         0.8897         0.8940


De la tabla anterior se observa que LR + TF obtiene el mejor desempeño global en todas las métricas. Sin embargo, LR + TF-IDF también presenta unas métricas bastante buenas y cercanas al mejor modelo.

También es posible notar que parece ser que el uso de TF-IDF presenta un impacto positivo especialmente para Naive Bayes.

### Validación cruzada

#### Cross validation Naive Bayes

En este experimento se implementa un proceso de validación cruzada de diez pliegues. El procedimiento consiste en construir dos configuraciones de *pipeline* que difieren únicamente en el tipo de representación vectorial utilizada para los documentos (frecuencias absolutas de términos y frecuencias ponderadas mediante TF-IDF) y medir su rendimiento de forma consistente a través de múltiples métricas de evaluación.

In [14]:
print(f"\n{'='*80}")
print("II. INVESTIGACIÓN DE VALIDACIÓN CRUZADA")
print(f"{'='*80}")
print("\n Preparando validación cruzada con train+validation...")
X_trainval = list(X_train) + list(X_val)
y_trainval = list(y_train) + list(y_val)

print(f"   - Datos para CV: {len(X_trainval):,} documentos")

# Configurar CV
cv_strategy = StratifiedKFold(n_splits=10, shuffle=True, random_state=RANDOM_STATE)
scoring_metrics = ["accuracy", "precision_macro", "recall_macro", "f1_macro", 
                  "precision_micro", "recall_micro", "f1_micro"]

# CV para Naive Bayes
print(f"\n{'='*60}")
print(" VALIDACIÓN CRUZADA - NAIVE BAYES")
print(f"{'='*60}")

nb_results = {}
for vectorizer_name, vectorizer_type in [("TF", "tf"), ("TF-IDF", "tfidf")]:
    print(f"\n  Evaluando NB + {vectorizer_name}...")
    
    pipeline_nb = Pipeline([
        ("vec", construir_vectorizador(vectorizer_type, USE_ENGLISH_STOPWORDS, optimized=False)),
        ("clf", MultinomialNB())
    ])
    
    cv_results = cross_validate(
        pipeline_nb, X_trainval, y_trainval,
        cv=cv_strategy, scoring=scoring_metrics, n_jobs=-1
    )
    
    nb_results[f"NB + {vectorizer_name}"] = cv_results
    
    print(f" Resultados NB + {vectorizer_name} (10-fold CV):")
    for metric in scoring_metrics:
        scores = cv_results[f"test_{metric}"]
        print(f"   {metric:15s}: {np.mean(scores):.4f} ± {np.std(scores):.4f}")



II. INVESTIGACIÓN DE VALIDACIÓN CRUZADA

 Preparando validación cruzada con train+validation...
   - Datos para CV: 13,179 documentos

 VALIDACIÓN CRUZADA - NAIVE BAYES

  Evaluando NB + TF...
 Resultados NB + TF (10-fold CV):
   accuracy       : 0.8582 ± 0.0095
   precision_macro: 0.8738 ± 0.0090
   recall_macro   : 0.8554 ± 0.0091
   f1_macro       : 0.8434 ± 0.0098
   precision_micro: 0.8582 ± 0.0095
   recall_micro   : 0.8582 ± 0.0095
   f1_micro       : 0.8582 ± 0.0095

  Evaluando NB + TF-IDF...
 Resultados NB + TF-IDF (10-fold CV):
   accuracy       : 0.8799 ± 0.0066
   precision_macro: 0.8898 ± 0.0071
   recall_macro   : 0.8684 ± 0.0068
   f1_macro       : 0.8680 ± 0.0074
   precision_micro: 0.8799 ± 0.0066
   recall_micro   : 0.8799 ± 0.0066
   f1_micro       : 0.8799 ± 0.0066


Los resultados de la validación cruzada muestran un patrón claro: el uso de representaciones TF-IDF ofrece una mejora consistente frente a las representaciones basadas únicamente en frecuencias absolutas (TF). En particular, Naive Bayes con TF-IDF alcanza un *accuracy* promedio cercano al 88% con una desviación estándar baja, lo que refleja tanto un mejor rendimiento como una mayor estabilidad entre los pliegues. Además, las métricas macro (precisión, recall y F1) son superiores en este esquema, lo que indica un tratamiento más equilibrado de las diferentes clases. En contraste, el modelo con TF presenta un rendimiento aceptable pero con un sesgo más marcado, evidenciado en un F1 macro más bajo.


#### Cross validation Logistic Regression

En este caso se lleva a cabo una búsqueda sistemática de hiperparámetros para modelos de regresión logística aplicados a clasificación de texto. El procedimiento consiste en integrar un *pipeline* con dos variantes de vectorización (TF y TF-IDF) y un clasificador de regresión logística, sobre el cual se exploran combinaciones de hiperparámetros clave: el coeficiente de regularización (*C*), el tipo de penalización (*l1* o *l2*) y el solucionador (*liblinear*, compatible con ambas penalizaciones). Para cada configuración se ejecuta una búsqueda en malla (*Grid Search*) con validación cruzada  de diez pliegues, utilizando como métrica principal el F1 macro.


In [15]:
print(f"\n{'='*60}")
print("BÚSQUEDA DE HIPERPARÁMETROS - LOGISTIC REGRESSION")
print(f"{'='*60}")

# Parámetros más amplios pero computacionalmente factibles
param_grid_lr = {
    'clf__C': [0.1, 0.5, 1.0, 2.0, 5.0],  # Regularización
    'clf__penalty': ['l1', 'l2'],          # Tipo de regularización
    'clf__solver': ['liblinear']            # Solver compatible con l1 y l2
}

lr_results = {}
for vectorizer_name, vectorizer_type in [("TF", "tf"), ("TF-IDF", "tfidf")]:
    print(f"\n Búsqueda de hiperparámetros LR + {vectorizer_name}...")
    
    pipeline_lr = Pipeline([
        ("vec", construir_vectorizador(vectorizer_type, USE_ENGLISH_STOPWORDS, optimized=False)),
        ("clf", LogisticRegression(multi_class="ovr", random_state=RANDOM_STATE, max_iter=3000))
    ])
    
    # GridSearch con CV
    grid_search = GridSearchCV(
        pipeline_lr, param_grid_lr,
        cv=cv_strategy, scoring='f1_macro',
        n_jobs=-1, verbose=1, refit=True
    )
    
    grid_search.fit(X_trainval, y_trainval)
    
    # Mejores parámetros
    print(f" Mejores parámetros LR + {vectorizer_name}:")
    for param, value in grid_search.best_params_.items():
        print(f"   {param}: {value}")
    print(f"   Mejor F1-macro CV: {grid_search.best_score_:.4f}")
    
    # Evaluación completa del mejor modelo
    best_pipeline = grid_search.best_estimator_
    cv_results_best = cross_validate(
        best_pipeline, X_trainval, y_trainval,
        cv=cv_strategy, scoring=scoring_metrics, n_jobs=-1
    )
    
    lr_results[f"LR + {vectorizer_name}"] = {
        'best_params': grid_search.best_params_,
        'best_score': grid_search.best_score_,
        'cv_results': cv_results_best,
        'best_pipeline': best_pipeline
    }
    
    print(f"📊 Resultados LR + {vectorizer_name} (mejor modelo, 10-fold CV):")
    for metric in scoring_metrics:
        scores = cv_results_best[f"test_{metric}"]
        print(f"   {metric:15s}: {np.mean(scores):.4f} ± {np.std(scores):.4f}")



BÚSQUEDA DE HIPERPARÁMETROS - LOGISTIC REGRESSION

 Búsqueda de hiperparámetros LR + TF...
Fitting 10 folds for each of 10 candidates, totalling 100 fits


c:\ProgramData\anaconda3\envs\ir-gensim\lib\site-packages\sklearn\linear_model\_logistic.py:1281: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.8. Use OneVsRestClassifier(LogisticRegression(..)) instead. Leave it to its default value to avoid this warning.
  warnings.warn(
c:\ProgramData\anaconda3\envs\ir-gensim\lib\site-packages\sklearn\linear_model\_logistic.py:1296: FutureWarning: Using the 'liblinear' solver for multiclass classification is deprecated. An error will be raised in 1.8. Either use another solver which supports the multinomial loss or wrap the estimator in a OneVsRestClassifier to keep applying a one-versus-rest scheme.
  warnings.warn(


 Mejores parámetros LR + TF:
   clf__C: 0.5
   clf__penalty: l2
   clf__solver: liblinear
   Mejor F1-macro CV: 0.8927
📊 Resultados LR + TF (mejor modelo, 10-fold CV):
   accuracy       : 0.8941 ± 0.0082
   precision_macro: 0.8963 ± 0.0083
   recall_macro   : 0.8915 ± 0.0084
   f1_macro       : 0.8927 ± 0.0084
   precision_micro: 0.8941 ± 0.0082
   recall_micro   : 0.8941 ± 0.0082
   f1_micro       : 0.8941 ± 0.0082

 Búsqueda de hiperparámetros LR + TF-IDF...
Fitting 10 folds for each of 10 candidates, totalling 100 fits


c:\ProgramData\anaconda3\envs\ir-gensim\lib\site-packages\sklearn\linear_model\_logistic.py:1281: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.8. Use OneVsRestClassifier(LogisticRegression(..)) instead. Leave it to its default value to avoid this warning.
  warnings.warn(
c:\ProgramData\anaconda3\envs\ir-gensim\lib\site-packages\sklearn\linear_model\_logistic.py:1296: FutureWarning: Using the 'liblinear' solver for multiclass classification is deprecated. An error will be raised in 1.8. Either use another solver which supports the multinomial loss or wrap the estimator in a OneVsRestClassifier to keep applying a one-versus-rest scheme.
  warnings.warn(


 Mejores parámetros LR + TF-IDF:
   clf__C: 5.0
   clf__penalty: l2
   clf__solver: liblinear
   Mejor F1-macro CV: 0.9084
📊 Resultados LR + TF-IDF (mejor modelo, 10-fold CV):
   accuracy       : 0.9105 ± 0.0062
   precision_macro: 0.9118 ± 0.0063
   recall_macro   : 0.9071 ± 0.0065
   f1_macro       : 0.9084 ± 0.0064
   precision_micro: 0.9105 ± 0.0062
   recall_micro   : 0.9105 ± 0.0062
   f1_micro       : 0.9105 ± 0.0062


Los resultados obtenidos muestran que la combinación de regresión logística con representación TF-IDF ofrece un desempeño muy sólido y balanceado en la tarea de clasificación. El mejor modelo, configurado con __regularización L2__ y un coeficiente de regularización de __5.0__, alcanza un F1-macro promedio de __0.9084__ en validación cruzada, lo que indica un buen equilibrio entre precisión y exhaustividad en todas las clases. Además, la consistencia de las métricas —con desviaciones estándar muy bajas en torno a 0.006— refleja una alta estabilidad del modelo frente a las distintas particiones de los datos.


#### Resumen de la validación cruzada

In [17]:
print(f"\n{'='*80}")
print(" RESUMEN VALIDACIÓN CRUZADA (10-fold)")
print(f"{'='*80}")

cv_summary = {}
for model_name, results in nb_results.items():
    cv_summary[model_name] = {
        'F1 Macro': f"{np.mean(results['test_f1_macro']):.4f} ± {np.std(results['test_f1_macro']):.4f}",
        'F1 Micro': f"{np.mean(results['test_f1_micro']):.4f} ± {np.std(results['test_f1_micro']):.4f}",
        'Accuracy': f"{np.mean(results['test_accuracy']):.4f} ± {np.std(results['test_accuracy']):.4f}"
    }

for model_name, results in lr_results.items():
    cv_summary[model_name] = {
        'F1 Macro': f"{np.mean(results['cv_results']['test_f1_macro']):.4f} ± {np.std(results['cv_results']['test_f1_macro']):.4f}",
        'F1 Micro': f"{np.mean(results['cv_results']['test_f1_micro']):.4f} ± {np.std(results['cv_results']['test_f1_micro']):.4f}",
        'Accuracy': f"{np.mean(results['cv_results']['test_accuracy']):.4f} ± {np.std(results['cv_results']['test_accuracy']):.4f}"
    }

df_cv = pd.DataFrame(cv_summary).T
print(df_cv)


 RESUMEN VALIDACIÓN CRUZADA (10-fold)
                    F1 Macro         F1 Micro         Accuracy
NB + TF      0.8434 ± 0.0098  0.8582 ± 0.0095  0.8582 ± 0.0095
NB + TF-IDF  0.8680 ± 0.0074  0.8799 ± 0.0066  0.8799 ± 0.0066
LR + TF      0.8927 ± 0.0084  0.8941 ± 0.0082  0.8941 ± 0.0082
LR + TF-IDF  0.9084 ± 0.0064  0.9105 ± 0.0062  0.9105 ± 0.0062


#### Evaluación de los modelos Naive Bayes enm test

In [18]:
print(f"\n{'='*80}")
print("III. EVALUACIÓN FINAL EN TEST SET")
print(f"{'='*80}")

print(" Evaluando todos los modelos en el conjunto de test...")

# Re-entrenar modelos con train+val y evaluar en test
final_results = {}

# Naive Bayes models
for vectorizer_name, vectorizer_type in [("TF", "tf"), ("TF-IDF", "tfidf")]:
    model_name = f"NB + {vectorizer_name}"
    print(f"\n  Evaluando {model_name} en test...")
    
    pipeline = Pipeline([
        ("vec", construir_vectorizador(vectorizer_type, USE_ENGLISH_STOPWORDS, optimized=True)),
        ("clf", MultinomialNB())
    ])
    
    pipeline.fit(X_trainval, y_trainval)
    y_pred = pipeline.predict(X_test)
    
    # Calcular métricas
    acc = accuracy_score(y_test, y_pred)
    p_macro, r_macro, f1_macro, _ = precision_recall_fscore_support(y_test, y_pred, average="macro")
    p_micro, r_micro, f1_micro, _ = precision_recall_fscore_support(y_test, y_pred, average="micro")
    
    final_results[model_name] = {
        'Accuracy': acc, 'Precision Macro': p_macro, 'Recall Macro': r_macro, 'F1 Macro': f1_macro,
        'Precision Micro': p_micro, 'Recall Micro': r_micro, 'F1 Micro': f1_micro
    }
    
    print(f"   Accuracy: {acc:.4f} | F1-Macro: {f1_macro:.4f} | F1-Micro: {f1_micro:.4f}")



III. EVALUACIÓN FINAL EN TEST SET
 Evaluando todos los modelos en el conjunto de test...

  Evaluando NB + TF en test...
   Accuracy: 0.8727 | F1-Macro: 0.8610 | F1-Micro: 0.8727

  Evaluando NB + TF-IDF en test...
   Accuracy: 0.8911 | F1-Macro: 0.8801 | F1-Micro: 0.8911


#### Evaluación de los modelos de regresión logística con mejores hiperparámetros en test

In [19]:
for vectorizer_name in ["TF", "TF-IDF"]:
    model_name = f"LR + {vectorizer_name}"
    print(f"\n  Evaluando {model_name} en test (mejores hiperparámetros)...")
    
    best_pipeline = lr_results[model_name]['best_pipeline']
    best_pipeline.fit(X_trainval, y_trainval)
    y_pred = best_pipeline.predict(X_test)
    
    # Calcular métricas
    acc = accuracy_score(y_test, y_pred)
    p_macro, r_macro, f1_macro, _ = precision_recall_fscore_support(y_test, y_pred, average="macro")
    p_micro, r_micro, f1_micro, _ = precision_recall_fscore_support(y_test, y_pred, average="micro")
    
    final_results[model_name] = {
        'Accuracy': acc, 'Precision Macro': p_macro, 'Recall Macro': r_macro, 'F1 Macro': f1_macro,
        'Precision Micro': p_micro, 'Recall Micro': r_micro, 'F1 Micro': f1_micro
    }
    
    print(f"   Accuracy: {acc:.4f} | F1-Macro: {f1_macro:.4f} | F1-Micro: {f1_micro:.4f}")
    print(f"   Mejores parámetros: {lr_results[model_name]['best_params']}")


  Evaluando LR + TF en test (mejores hiperparámetros)...


c:\ProgramData\anaconda3\envs\ir-gensim\lib\site-packages\sklearn\linear_model\_logistic.py:1281: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.8. Use OneVsRestClassifier(LogisticRegression(..)) instead. Leave it to its default value to avoid this warning.
  warnings.warn(
c:\ProgramData\anaconda3\envs\ir-gensim\lib\site-packages\sklearn\linear_model\_logistic.py:1296: FutureWarning: Using the 'liblinear' solver for multiclass classification is deprecated. An error will be raised in 1.8. Either use another solver which supports the multinomial loss or wrap the estimator in a OneVsRestClassifier to keep applying a one-versus-rest scheme.
  warnings.warn(


   Accuracy: 0.8915 | F1-Macro: 0.8898 | F1-Micro: 0.8915
   Mejores parámetros: {'clf__C': 0.5, 'clf__penalty': 'l2', 'clf__solver': 'liblinear'}

  Evaluando LR + TF-IDF en test (mejores hiperparámetros)...


c:\ProgramData\anaconda3\envs\ir-gensim\lib\site-packages\sklearn\linear_model\_logistic.py:1281: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.8. Use OneVsRestClassifier(LogisticRegression(..)) instead. Leave it to its default value to avoid this warning.
  warnings.warn(
c:\ProgramData\anaconda3\envs\ir-gensim\lib\site-packages\sklearn\linear_model\_logistic.py:1296: FutureWarning: Using the 'liblinear' solver for multiclass classification is deprecated. An error will be raised in 1.8. Either use another solver which supports the multinomial loss or wrap the estimator in a OneVsRestClassifier to keep applying a one-versus-rest scheme.
  warnings.warn(


   Accuracy: 0.9106 | F1-Macro: 0.9087 | F1-Micro: 0.9106
   Mejores parámetros: {'clf__C': 5.0, 'clf__penalty': 'l2', 'clf__solver': 'liblinear'}


In [20]:
print(f"\n{'='*80}")
print(" RESULTADOS FINALES EN TEST SET")
print(f"{'='*80}")

df_final = pd.DataFrame(final_results).T
print("\n Métricas de rendimiento:")
print(df_final.round(4))

# Identificar el mejor modelo
best_model_f1_macro = df_final['F1 Macro'].idxmax()
best_model_f1_micro = df_final['F1 Micro'].idxmax()
best_model_accuracy = df_final['Accuracy'].idxmax()

print(f"\n MEJORES MODELOS:")
print(f"    Mejor F1-Macro:  {best_model_f1_macro} ({df_final.loc[best_model_f1_macro, 'F1 Macro']:.4f})")
print(f"    Mejor F1-Micro:  {best_model_f1_micro} ({df_final.loc[best_model_f1_micro, 'F1 Micro']:.4f})")
print(f"    Mejor Accuracy:  {best_model_accuracy} ({df_final.loc[best_model_accuracy, 'Accuracy']:.4f})")



 RESULTADOS FINALES EN TEST SET

 Métricas de rendimiento:
             Accuracy  Precision Macro  Recall Macro  F1 Macro  \
NB + TF        0.8727           0.8833        0.8694    0.8610   
NB + TF-IDF    0.8911           0.9010        0.8799    0.8801   
LR + TF        0.8915           0.8916        0.8887    0.8898   
LR + TF-IDF    0.9106           0.9112        0.9074    0.9087   

             Precision Micro  Recall Micro  F1 Micro  
NB + TF               0.8727        0.8727    0.8727  
NB + TF-IDF           0.8911        0.8911    0.8911  
LR + TF               0.8915        0.8915    0.8915  
LR + TF-IDF           0.9106        0.9106    0.9106  

 MEJORES MODELOS:
    Mejor F1-Macro:  LR + TF-IDF (0.9087)
    Mejor F1-Micro:  LR + TF-IDF (0.9106)
    Mejor Accuracy:  LR + TF-IDF (0.9106)


In [21]:
print(f"\n{'='*80}")
print(" ANÁLISIS DEL MEJOR MODELO")
print(f"{'='*80}")

best_overall = best_model_f1_macro  # Usar F1-macro como criterio principal
print(f"Modelo seleccionado: {best_overall}")

if best_overall.startswith("LR"):
    vectorizer_type = "TF" if "TF-IDF" not in best_overall else "TF-IDF"
    best_params = lr_results[best_overall]['best_params']
    print(f"Representación: {vectorizer_type}")
    print(f"Hiperparámetros optimizados:")
    for param, value in best_params.items():
        print(f"   {param}: {value}")

print(f"\nReporte detallado del mejor modelo:")
if best_overall.startswith("LR"):
    best_pipeline = lr_results[best_overall]['best_pipeline']
else:
    vectorizer_type = "tf" if "TF-IDF" not in best_overall else "tfidf"
    best_pipeline = Pipeline([
        ("vec", construir_vectorizador(vectorizer_type, USE_ENGLISH_STOPWORDS, optimized=True)),
        ("clf", MultinomialNB())
    ])
    best_pipeline.fit(X_trainval, y_trainval)

y_pred_best = best_pipeline.predict(X_test)
print(classification_report(y_test, y_pred_best, target_names=target_names, digits=3))


 ANÁLISIS DEL MEJOR MODELO
Modelo seleccionado: LR + TF-IDF
Representación: TF-IDF
Hiperparámetros optimizados:
   clf__C: 5.0
   clf__penalty: l2
   clf__solver: liblinear

Reporte detallado del mejor modelo:
                          precision    recall  f1-score   support

             alt.atheism      0.893     0.900     0.896       240
           comp.graphics      0.816     0.863     0.839       292
 comp.os.ms-windows.misc      0.828     0.861     0.844       296
comp.sys.ibm.pc.hardware      0.830     0.797     0.813       295
   comp.sys.mac.hardware      0.902     0.899     0.901       288
          comp.windows.x      0.891     0.891     0.891       294
            misc.forsale      0.845     0.894     0.869       292
               rec.autos      0.929     0.923     0.926       297
         rec.motorcycles      0.973     0.956     0.964       298
      rec.sport.baseball      0.970     0.980     0.975       298
        rec.sport.hockey      0.973     0.970     0.972       

Al final de toda la experimentación, el mejor modelo identificado fue la regresión logística con representación TF-IDF y regularización L2, que alcanzó un desempeño sobresaliente con una exactitud global del 91.1 %. El análisis detallado por clase muestra resultados consistentes y elevados en precisión, recall y F1, con un promedio macro de 0.911, 0.907 y 0.909, respectivamente. Se destacan categorías como *rec.sport.baseball*, *rec.motorcycles* y *talk.politics.mideast*, donde los puntajes F1 superan el 0.95, evidenciando la capacidad del modelo para distinguir con gran efectividad estos temas. Aunque algunas clases como *talk.religion.misc* presentan un rendimiento relativamente menor, el modelo mantiene en general un balance adecuado en todos los grupos.
